In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/pharmahack23/"

In [96]:
import matplotlib.pyplot as plt
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import os
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier, VotingClassifier
from sklearn.feature_selection import RFE, SelectKBest, chi2, SelectFromModel
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, matthews_corrcoef
from sklearn.model_selection import LeaveOneOut, ShuffleSplit, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb

import random

random_seed = 0

def seed_everything(seed):
    """"
    Seed everything.
    """   
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)
#     torch.cuda.manual_seed_all(seed)
#     torch.backends.cudnn.deterministic = True

seed_everything(seed=random_seed)

import warnings
warnings.filterwarnings('ignore')

Hot Encoding And Min-Max Scaler

In [97]:
df = pd.read_excel(path + "train_data.xlsx", sheet_name="x")
df_y = pd.read_excel(path + "train_data.xlsx", sheet_name="y")
df["y"] = df_y["dd10 CM Content"].values
df.loc[df['y'] > 90, "y_cat"] = 1
df.loc[df['y'] <= 90, "y_cat"] = 0

df_y = df["y_cat"]
df.drop(labels=['y_cat', 'y'], axis=1, inplace=True)

# df, df_y = smote.fit_resample(df, df_y)
# print(df.shape, df_y.shape)


# normalized df
df_x = df
scaler = MinMaxScaler()
scaler.fit(df)

MinMaxScaler()

Ensemble based Feature Selection; Taking common features among the 4 methods which is implemented as, 

1.   Pearson's correlation
2.   Chi-Squared
3.   Recursive Feature Elimination
4.   Lasso



In [98]:
#Pearson's correlation
def cor_selector(X, y, num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:, np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature


num_feats = 50
cor_support, cor_feature = cor_selector(df_x, df_y, num_feats)
print(str(len(cor_feature)), 'selected features')
print(cor_feature)

# Chi-Squared


X_norm = MinMaxScaler().fit_transform(df_x)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, df_y)
chi_support = chi_selector.get_support()
chi_feature = df_x.loc[:, chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')
print(chi_feature)

# Recursive Feature Elimination


rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, df_y)
rfe_support = rfe_selector.get_support()
rfe_feature = df_x.loc[:, rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
print(rfe_feature)

# Lasso: SelectFromModel

embeded_lr_selector = SelectFromModel(LogisticRegression(penalty="l2"), max_features=num_feats)
embeded_lr_selector.fit(X_norm, df_y)

embeded_lr_support = embeded_lr_selector.get_support()
embeded_lr_feature = df_x.loc[:, embeded_lr_support].columns.tolist()
print(str(len(embeded_lr_feature)), 'selected features')
print(embeded_lr_feature)

# Tree-based: SelectFromModel

embeded_rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100), max_features=num_feats)
embeded_rf_selector.fit(df_x, df_y)

embeded_rf_support = embeded_rf_selector.get_support()
embeded_rf_feature = df_x.loc[:, embeded_rf_support].columns.tolist()
print(str(len(embeded_rf_feature)), 'selected features')
print(embeded_rf_feature)

50 selected features
['dd1 Aggregate Size ', 'd1 Average pH Gradient', 'dd1 Lactate Concentration', 'Average DO concentration dd6', 'd0 Average pH Gradient', 'dd0-dd1 Aggregate Size Gradient', 'DO gradient/cell count dd1', 'Average DO concentration d1', 'dd5 Lactate Concentration', 'dd7 Glucose Concentration', 'dd5 DO 2nd derivative/cell count', 'dd5 Average of 2nd derivative DO', 'Average DO concentration dd7', 'DO concentration/cell count dd5', 'dd5 Aggregate Size ', 'dd0 Lactate Concentration', 'dd3 Average pH Gradient', 'Average DO concentration gradient dd5', 'dd7 Average pH Gradient', 'DO concentration/cell count dd2', 'dd7 Average pH', 'dd0-dd1 Cell Density Gradient', 'dd5 Glucose Concentration', 'DO concentration/cell count dd1', 'DO gradient/cell count dd0', 'dd3 Average of 2nd derivative DO', 'dd2-dd3 Cell Density Gradient', 'dd3 Lactate Concentration', 'Average DO concentration gradient dd1', 'dd3 DO 2nd derivative/cell count', 'Average DO concentration gradient dd0', 'dd3 G

Feature Selection ( Union and Intersection )

In [99]:
total_feature = [cor_feature, chi_feature, rfe_feature, embeded_lr_feature]


def intersect(*d):
    sets = iter(map(set, d))
    result = sets.next()
    for s in sets:
        result = result.intersection(s)
    return result


features_union = list(set(cor_feature + chi_feature + rfe_feature + embeded_lr_feature))
features_intersection = list(set.intersection(*map(set, total_feature)))

In [100]:
intersec = True
if intersec:
    features = features_intersection
else:
    features = features_union

df_train_x = pd.read_excel(path + "train_data.xlsx", sheet_name="x")

df_train_y = pd.read_excel(path + "train_data.xlsx", sheet_name="y")
df_train_x["y"] = df_train_y["dd10 CM Content"].values
df_train_x.loc[df_train_x['y'] > 90, "y_cat"] = 1
df_train_x.loc[df_train_x['y'] <= 90, "y_cat"] = 0

df_train_y = df_train_x["y_cat"]
df_train_x.drop(labels=['y_cat', 'y'], axis=1, inplace=True)

# df_train_x, df_train_y = smote.fit_resample(df_train_x, df_train_y)
# print(df_train_x.shape, df_train_y.shape)


df_train_x = df_train_x[features]

# normalized df
scaler = MinMaxScaler()
df_tr = df_train_x
scaler.fit(df_tr)
df_train_x = scaler.transform(df_train_x)

df_test_x = pd.read_excel(path + "test_data_corrected.xlsx", sheet_name="x")
df_test_y = pd.read_excel(path + "test_data_corrected.xlsx", sheet_name="y")
df_test_x["y"] = df_test_y["dd10 CM Content"].values
df_test_x.loc[df_test_x['y'] > 90, "y_cat"] = 1
df_test_x.loc[df_test_x['y'] <= 90, "y_cat"] = 0

df_test_y = df_test_x["y_cat"]
df_test_x.drop(labels=['y_cat', 'y'], axis=1, inplace=True)
# normalized df

df_test_x = df_test_x[features]

df_test_x = scaler.transform(df_test_x)

print(df_train_x.shape, df_train_y.shape, df_test_x.shape, df_test_y.shape)

(42, 31) (42,) (18, 31) (18,)


Leave One Out Implementation

In [101]:
loo = LeaveOneOut()
loo.get_n_splits(df_train_x)

42

In [102]:
print("#######################################################################")
print("######################STANDALONE MACHINE LEARNING MODELS###############")
print("#######################################################################")



avg_accuracy = []
avg_mcc = []
list_of_models = []
for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    lr = LogisticRegression(random_state=random_seed, class_weight="balanced")
    lr.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_lr = lr.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_lr))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_lr))
    list_of_models.append(lr)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_lr = list_of_models[max_acc_index].predict(df_test_x)
print("Test Accuracy score for LogisticRegression on Best LOO", round(accuracy_score(df_test_y, y_pred_lr), 3))
print("Test Precision score for LogisticRegression on Best LOO", round(precision_score(df_test_y, y_pred_lr), 3))
print("Test Recall score for LogisticRegression on Best LOO", round(recall_score(df_test_y, y_pred_lr), 3))

print("MCC score for LogisticRegression on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_lr), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    svc = SVC(random_state=random_seed, degree=25, class_weight="balanced")
    svc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_svc = svc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_svc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_svc))
    list_of_models.append(svc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_svc = list_of_models[max_acc_index].predict(df_test_x)
print("Test Accuracy score for SVC on Best LOO", round(accuracy_score(df_test_y, y_pred_svc), 3))
print("Test Precision score for SVC on Best LOO", round(precision_score(df_test_y, y_pred_svc), 3))
print("Test Recall score for SVC on Best LOO", round(recall_score(df_test_y, y_pred_svc), 3))
print("MCC score for SVC on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_svc), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    gnb = GaussianNB()
    gnb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_gnb = gnb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_gnb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_gnb))
    list_of_models.append(gnb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_gnb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for GaussianNB on Best LOO", round(accuracy_score(df_test_y, y_pred_gnb), 3))
print("Test Precision score for GaussianNB on Best LOO", round(precision_score(df_test_y, y_pred_gnb), 3))
print("Test Recall score for GaussianNB on Best LOO", round(recall_score(df_test_y, y_pred_gnb), 3))
print("MCC score for GaussianNB on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_gnb), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    mnb = MultinomialNB()
    mnb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_mnb = mnb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_mnb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_mnb))
    list_of_models.append(mnb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_mnb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for MultinomialNB on Best LOO", round(accuracy_score(df_test_y, y_pred_mnb), 3))
print("Test Precision score for MultinomialNB on Best LOO", round(precision_score(df_test_y, y_pred_mnb), 3))
print("Test Recall score for MultinomialNB on Best LOO", round(recall_score(df_test_y, y_pred_mnb), 3))
print("MCC score for MultinomialNB on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_mnb), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    sgd = SGDClassifier(random_state=random_seed, loss="log_loss", n_jobs=-1, class_weight="balanced")
    sgd.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_sgd = sgd.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_sgd))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_sgd))
    list_of_models.append(sgd)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_sgd = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for SGDClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_sgd), 3))
print("Test Precision score for SGDClassifier on Best LOO", round(precision_score(df_test_y, y_pred_sgd), 3))
print("Test Recall score for SGDClassifier on Best LOO", round(recall_score(df_test_y, y_pred_sgd), 3))
print("MCC score for SGDClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_sgd), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    knn = KNeighborsClassifier(weights="distance", n_jobs=-1)
    knn.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_knn = knn.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_knn))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_knn))
    list_of_models.append(knn)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_knn = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for KNeighborsClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_knn), 3))
print("Test Precision score for KNeighborsClassifier on Best LOO", round(precision_score(df_test_y, y_pred_knn), 3))
print("Test Recall score for KNeighborsClassifier on Best LOO", round(recall_score(df_test_y, y_pred_knn), 3))
print("MCC score for KNeighborsClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_knn), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    dtc = DecisionTreeClassifier(random_state=random_seed, criterion="entropy", class_weight="balanced")
    dtc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_dtc = dtc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_dtc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_dtc))
    list_of_models.append(dtc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_dtc = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for DecisionTreeClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_dtc), 3))
print("Test Precision score for DecisionTreeClassifier on Best LOO", round(precision_score(df_test_y, y_pred_dtc), 3))
print("Test Recall score for DecisionTreeClassifier on Best LOO", round(recall_score(df_test_y, y_pred_dtc), 3))
print("MCC score for DecisionTreeClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_dtc), 3))
print()

print("#######################################################################")
print("######################ENSEMBLE BASED LEARNING MODELS##################")
print("#######################################################################")

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    rfc = RandomForestClassifier(random_state=random_seed, max_depth=6, n_estimators = 200, n_jobs=-1, class_weight="balanced")
    rfc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_rfc = rfc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_rfc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_rfc))
    list_of_models.append(rfc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_rfc = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for RandomForestClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_rfc), 3))
print("Test Precision score for RandomForestClassifier on Best LOO", round(precision_score(df_test_y, y_pred_rfc), 3))
print("Test Recall score for RandomForestClassifier on Best LOO", round(recall_score(df_test_y, y_pred_rfc), 3))
print("MCC score for RandomForestClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_rfc), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    gbc = GradientBoostingClassifier(random_state=random_seed, max_depth=6, n_estimators = 200)
    gbc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_gbc = gbc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_gbc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_gbc))
    list_of_models.append(gbc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_gbc = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for GradientBoostingClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_gbc), 3))
print("Test Precision score for GradientBoostingClassifier on Best LOO", round(precision_score(df_test_y, y_pred_gbc), 3))
print("Test Recall score for GradientBoostingClassifier on Best LOO", round(recall_score(df_test_y, y_pred_gbc), 3))
print("MCC score for GradientBoostingClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_gbc), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    xgb = XGBClassifier(random_state=random_seed, max_depth=7, n_estimators = 200)
    xgb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_xgb = xgb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_xgb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_xgb))
    list_of_models.append(xgb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_xgb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for XGBClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_xgb), 3))
print("Test Precision score for XGBClassifier on Best LOO", round(precision_score(df_test_y, y_pred_xgb), 3))
print("Test Recall score for XGBClassifier on Best LOO", round(recall_score(df_test_y, y_pred_xgb), 3))
print("MCC score for XGBClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_xgb), 3))
print()


avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    adgb = AdaBoostClassifier(random_state=random_seed, n_estimators = 200)
    adgb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_adgb = adgb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_adgb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_adgb))
    list_of_models.append(xgb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_adgb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for AdaBoostClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_adgb), 3))
print("Test Precision score for AdaBoostClassifier on Best LOO", round(precision_score(df_test_y, y_pred_adgb), 3))
print("Test Recall score for AdaBoostClassifier on Best LOO", round(recall_score(df_test_y, y_pred_adgb), 3))
print("MCC score for AdaBoostClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_adgb), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
    lgbm_c = lgb.LGBMClassifier(random_state=random_seed, n_estimators = 200, n_jobs=-1)
    lgbm_c.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_lgbm_c = lgbm_c.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_lgbm_c))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_lgbm_c))
    list_of_models.append(xgb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_lgbm_c = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for LGBMClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_lgbm_c), 3))
print("Test Precision score for LGBMClassifier on Best LOO", round(precision_score(df_test_y, y_pred_lgbm_c), 3))
print("Test Recall score for LGBMClassifier on Best LOO", round(recall_score(df_test_y, y_pred_lgbm_c), 3))
print("MCC score for LGBMClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_lgbm_c), 3))
print()


avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(loo.split(df_train_x)):
#     lr = LogisticRegression(random_state=random_seed)
#     svc = SVC(random_state=random_seed, degree=25)
#     gnb = GaussianNB()
#     xgb = XGBClassifier(random_state=random_seed)
#     eclf1 = VotingClassifier(estimators=[('lr', lr), ('svc', svc), ('gnb', gnb), ('xgb', xgb)], voting='hard')
    gbc = GradientBoostingClassifier(random_state=random_seed, max_depth=6, n_estimators = 200)
    dgb = AdaBoostClassifier(random_state=random_seed, n_estimators = 200)
    xgb = XGBClassifier(random_state=random_seed, max_depth=7, n_estimators = 200)
    eclf1 = VotingClassifier(estimators=[('gbc', gbc), ('adgb', adgb), ('xgb', xgb)], voting='hard')
    
    eclf1 = eclf1.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_eclf1 = eclf1.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_eclf1))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_eclf1))
    list_of_models.append(eclf1)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_eclf1 = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for VotingClassifier on Best LOO", round(accuracy_score(df_test_y, y_pred_eclf1), 3))
print("Test Precision score for VotingClassifier on Best LOO", round(precision_score(df_test_y, y_pred_eclf1), 3))
print("Test Recall score for VotingClassifier on Best LOO", round(recall_score(df_test_y, y_pred_eclf1), 3))
print("MCC score for VotingClassifier on Best LOO", round(matthews_corrcoef(df_test_y, y_pred_eclf1), 3))
print()


#######################################################################
######################STANDALONE MACHINE LEARNING MODELS###############
#######################################################################
Test Accuracy score for LogisticRegression on Best LOO 0.556
Test Precision score for LogisticRegression on Best LOO 0.385
Test Recall score for LogisticRegression on Best LOO 1.0
MCC score for LogisticRegression on Best LOO 0.385

Test Accuracy score for SVC on Best LOO 0.611
Test Precision score for SVC on Best LOO 0.4
Test Recall score for SVC on Best LOO 0.8
MCC score for SVC on Best LOO 0.305

Test Accuracy score for GaussianNB on Best LOO 0.333
Test Precision score for GaussianNB on Best LOO 0.294
Test Recall score for GaussianNB on Best LOO 1.0
MCC score for GaussianNB on Best LOO 0.15

Test Accuracy score for MultinomialNB on Best LOO 0.667
Test Precision score for MultinomialNB on Best LOO 0.4
Test Recall score for MultinomialNB on Best LOO 0.4
MCC score for Multin

Monte Carlo Implementation

In [103]:
rs = ShuffleSplit(n_splits=15, test_size=.25, random_state=random_seed)

In [104]:
print("#######################################################################")
print("######################STANDALONE MACHINE LEARNING MODELS###############")
print("#######################################################################")


avg_accuracy = []
avg_mcc = []
list_of_models = []
for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    lr = LogisticRegression(random_state=random_seed, class_weight="balanced")
    lr.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_lr = lr.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_lr))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_lr))
    list_of_models.append(lr)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_lr = list_of_models[max_acc_index].predict(df_test_x)
print("Test Accuracy score for LogisticRegression on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_lr), 3))
print("Test Precision score for LogisticRegression on Best MonteCarlo", round(precision_score(df_test_y, y_pred_lr), 3))
print("Test Recall score for LogisticRegression on Best MonteCarlo", round(recall_score(df_test_y, y_pred_lr), 3))
print("MCC score for LogisticRegression on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_lr), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    svc = SVC(random_state=random_seed, degree=25, class_weight="balanced")
    svc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_svc = svc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_svc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_svc))
    list_of_models.append(svc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_svc = list_of_models[max_acc_index].predict(df_test_x)
print("Test Accuracy score for SVC on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_svc), 3))
print("Test Precision score for SVC on Best MonteCarlo", round(precision_score(df_test_y, y_pred_svc), 3))
print("Test Recall score for SVC on Best MonteCarlo", round(recall_score(df_test_y, y_pred_svc), 3))
print("MCC score for SVC on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_svc), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    gnb = GaussianNB()
    gnb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_gnb = gnb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_gnb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_gnb))
    list_of_models.append(gnb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_gnb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for GaussianNB on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_gnb), 3))
print("Test Precision score for GaussianNB on Best MonteCarlo", round(precision_score(df_test_y, y_pred_gnb), 3))
print("Test Recall score for GaussianNB on Best MonteCarlo", round(recall_score(df_test_y, y_pred_gnb), 3))
print("MCC score for GaussianNB on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_gnb), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    mnb = MultinomialNB()
    mnb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_mnb = mnb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_mnb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_mnb))
    list_of_models.append(mnb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_mnb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for MultinomialNB on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_mnb), 3))
print("Test Precision score for MultinomialNB on Best MonteCarlo", round(precision_score(df_test_y, y_pred_mnb), 3))
print("Test Recall score for MultinomialNB on Best MonteCarlo", round(recall_score(df_test_y, y_pred_mnb), 3))
print("MCC score for MultinomialNB on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_mnb), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    sgd = SGDClassifier(random_state=random_seed, loss="log_loss", n_jobs=-1, class_weight="balanced")
    sgd.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_sgd = sgd.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_sgd))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_sgd))
    list_of_models.append(sgd)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_sgd = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for SGDClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_sgd), 3))
print("Test Precision score for SGDClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_sgd), 3))
print("Test Recall score for SGDClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_sgd), 3))
print("MCC score for SGDClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_sgd), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    knn = KNeighborsClassifier(weights="distance", n_jobs=-1)
    knn.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_knn = knn.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_knn))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_knn))
    list_of_models.append(knn)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_knn = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for KNeighborsClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_knn), 3))
print("Test Precision score for KNeighborsClassifier on Best MonteCarlo",round(precision_score(df_test_y, y_pred_knn), 3))
print("Test Recall score for KNeighborsClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_knn), 3))
print("MCC score for KNeighborsClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_knn), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    dtc = DecisionTreeClassifier(random_state=random_seed, criterion="entropy", class_weight="balanced")
    dtc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_dtc = dtc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_dtc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_dtc))
    list_of_models.append(dtc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_dtc = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for DecisionTreeClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_dtc), 3))
print("Test Precision score for DecisionTreeClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_dtc), 3))
print("Test Recall score for DecisionTreeClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_dtc), 3))
print("MCC score for DecisionTreeClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_dtc), 3))
print()

print("#######################################################################")
print("######################ENSEMBLE BASED LEARNING MODELS##################")
print("#######################################################################")


avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    rfc = RandomForestClassifier(random_state=random_seed, max_depth=6, n_estimators = 200, n_jobs=-1, class_weight="balanced")
    rfc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_rfc = rfc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_rfc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_rfc))
    list_of_models.append(rfc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_rfc = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for RandomForestClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_rfc), 3))
print("Test Precision score for RandomForestClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_rfc), 3))
print("Test Recall score for RandomForestClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_rfc), 3))
print("MCC score for RandomForestClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_rfc), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    gbc = GradientBoostingClassifier(random_state=random_seed, max_depth=6, n_estimators = 200)
    gbc.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_gbc = gbc.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_gbc))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_gbc))
    list_of_models.append(gbc)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_gbc = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for GradientBoostingClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_gbc), 3))
print("Test Precision score for GradientBoostingClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_gbc), 3))
print("Test Recall score for GradientBoostingClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_gbc), 3))
print("MCC score for GradientBoostingClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_gbc), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    xgb = XGBClassifier(random_state=random_seed, max_depth=7, n_estimators = 200)
    xgb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_xgb = xgb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_xgb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_xgb))
    list_of_models.append(xgb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_xgb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for XGBClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_xgb), 3))
print("Test Precision score for XGBClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_xgb), 3))
print("Test Recall score for XGBClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_xgb), 3))
print("MCC score for XGBClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_xgb), 3))
print()


avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    adgb = AdaBoostClassifier(random_state=random_seed, n_estimators = 200)
    adgb.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_adgb = adgb.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_adgb))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_adgb))
    list_of_models.append(xgb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_adgb = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for AdaBoostClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_adgb), 3))
print("Test Precision score for AdaBoostClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_adgb), 3))
print("Test Recall score for AdaBoostClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_adgb), 3))
print("MCC score for AdaBoostClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_adgb), 3))
print()

avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
    lgbm_c = lgb.LGBMClassifier(random_state=random_seed, n_estimators = 200, n_jobs=-1)
    lgbm_c.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_lgbm_c = lgbm_c.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_lgbm_c))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_lgbm_c))
    list_of_models.append(xgb)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_lgbm_c = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for LGBMClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_lgbm_c), 3))
print("Test Precision score for LGBMClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_lgbm_c), 3))
print("Test Recall score for LGBMClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_lgbm_c), 3))
print("MCC score for LGBMClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_lgbm_c), 3))
print()


avg_accuracy = []
avg_mcc = []
list_of_models = []

for i, (train_index, test_index) in enumerate(rs.split(df_train_x)):
#     lr = LogisticRegression(random_state=random_seed)
#     svc = SVC(random_state=random_seed, degree=25)
#     gnb = GaussianNB()
#     xgb = XGBClassifier(random_state=random_seed)
#     eclf1 = VotingClassifier(estimators=[('lr', lr), ('svc', svc), ('gnb', gnb), ('xgb', xgb)], voting='hard')
    gbc = GradientBoostingClassifier(random_state=random_seed, max_depth=6, n_estimators = 200)
    dgb = AdaBoostClassifier(random_state=random_seed, n_estimators = 200)
    xgb = XGBClassifier(random_state=random_seed, max_depth=7, n_estimators = 200)
    eclf1 = VotingClassifier(estimators=[('gbc', gbc), ('adgb', adgb), ('xgb', xgb)], voting='hard')
    
    eclf1 = eclf1.fit(df_train_x[train_index], df_train_y[train_index])
    y_pred_eclf1 = eclf1.predict(df_train_x[test_index])

    avg_accuracy.append(accuracy_score(df_train_y[test_index], y_pred_eclf1))
    avg_mcc.append(matthews_corrcoef(df_train_y[test_index], y_pred_eclf1))
    list_of_models.append(eclf1)

max_acc_index = avg_accuracy.index(max(avg_accuracy))
y_pred_eclf1 = list_of_models[max_acc_index].predict(df_test_x)

print("Test Accuracy score for VotingClassifier on Best MonteCarlo", round(accuracy_score(df_test_y, y_pred_eclf1), 3))
print("Test Precision score for VotingClassifier on Best MonteCarlo", round(precision_score(df_test_y, y_pred_eclf1), 3))
print("Test Recall score for VotingClassifier on Best MonteCarlo", round(recall_score(df_test_y, y_pred_eclf1), 3))
print("MCC score for VotingClassifier on Best MonteCarlo", round(matthews_corrcoef(df_test_y, y_pred_eclf1), 3))
print()



#######################################################################
######################STANDALONE MACHINE LEARNING MODELS###############
#######################################################################
Test Accuracy score for LogisticRegression on Best MonteCarlo 0.611
Test Precision score for LogisticRegression on Best MonteCarlo 0.417
Test Recall score for LogisticRegression on Best MonteCarlo 1.0
MCC score for LogisticRegression on Best MonteCarlo 0.439

Test Accuracy score for SVC on Best MonteCarlo 0.667
Test Precision score for SVC on Best MonteCarlo 0.444
Test Recall score for SVC on Best MonteCarlo 0.8
MCC score for SVC on Best MonteCarlo 0.372

Test Accuracy score for GaussianNB on Best MonteCarlo 0.333
Test Precision score for GaussianNB on Best MonteCarlo 0.294
Test Recall score for GaussianNB on Best MonteCarlo 1.0
MCC score for GaussianNB on Best MonteCarlo 0.15

Test Accuracy score for MultinomialNB on Best MonteCarlo 0.556
Test Precision score for Multinomi